#Load torchtext and initialize XLM-R model

In [1]:
import torch
import torch.nn as nn
import torchtext

from torchtext.models import RobertaClassificationHead
from torchtext.functional import to_tensor

xlmr_large = torchtext.models.XLMR_LARGE_ENCODER
classifier_head = torchtext.models.RobertaClassificationHead(num_classes=2, input_dim = 1024)
model = xlmr_large.get_model(head=classifier_head)

# Put model into inference mode (reduces runtime even without BT - esp for GPU execution, required for Better Transformer)
model.eval()

# Define input transform
transform = xlmr_large.transform()


Downloading: "https://download.pytorch.org/models/text/xlmr.large.encoder.pt" to /home/zlabs-hwa-01/.cache/torch/hub/checkpoints/xlmr.large.encoder.pt
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2.08G/2.08G [01:40<00:00, 22.3MB/s]
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5.07M/5.07M [00:00<00:00, 23.5MB/s]
Downloading: "https://download.pytorch.org/models/text/xlmr.vocab.pt" to /home/zlabs-hwa-01/.cache/torch/hub/checkpoints/xlmr.vocab.pt
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4.85M/4.85M [00:00<00:00, 9.93MB/s]


# System Information

In [2]:
import platform

DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

cpu = platform.processor()
gpu = torch.cuda.get_device_name(DEVICE)

print(f"torch version: {torch.__version__}")
print(f"torch cuda available: {torch.cuda.is_available()}")
print(f"CPU type: {cpu}")
print(f"GPU type: {gpu}")

/home/zlabs-hwa-01/anaconda3/lib/python3.10/site-packages/torch/cuda/__init__.py:107: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:109.)
  return torch._C._cuda_getDeviceCount() > 0


RuntimeError: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW

# Check default sparsity support setting
Sparsity support enables transformers to skip padding in inputs.

In [ ]:
model.encoder.transformer.layers.enable_nested_tensor

# Benchmark setup

###Define inputs

In [ ]:
small_input_batch = [
               "Hello world",
               "How are you!"
]
big_input_batch = [
               "Hello world",
               "How are you!",
               """`Well, Prince, so Genoa and Lucca are now just family estates of the
Buonapartes. But I warn you, if you don't tell me that this means war,
if you still try to defend the infamies and horrors perpetrated by
that Antichrist- I really believe he is Antichrist- I will have
nothing more to do with you and you are no longer my friend, no longer
my 'faithful slave,' as you call yourself! But how do you do? I see
I have frightened you- sit down and tell me all the news.`

It was in July, 1805, and the speaker was the well-known Anna
Pavlovna Scherer, maid of honor and favorite of the Empress Marya
Fedorovna. With these words she greeted Prince Vasili Kuragin, a man
of high rank and importance, who was the first to arrive at her
reception. Anna Pavlovna had had a cough for some days. She was, as
she said, suffering from la grippe; grippe being then a new word in
St. Petersburg, used only by the elite."""
]

###Select small or big input set

Modify the assignment to input_batch below to select either the small_input_batch or big_inoput_batch, or substitute your own inputs.

In [ ]:
input_batch=big_input_batch

model_input = to_tensor(transform(input_batch), padding_value=1)
output = model(model_input)
output.shape

###Iteration count for performance measurements

In [ ]:
ITERATIONS=10

#Measure CPU  performance with slow and fast path, without and with sparsity

Sparsity support enables transformers to skip padding in inputs.


### CPU performance without BT sparsity

In [ ]:
model.encoder.transformer.layers.enable_nested_tensor = False

In [ ]:
print("slow path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=False) as prof:
  for i in range(ITERATIONS):
    output = model(model_input)
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cpu_time_total", row_limit=5))

print("fast path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=False) as prof:
  with torch.no_grad():
    for i in range(ITERATIONS):
      output = model(model_input)
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cpu_time_total", row_limit=5))



###CPU performance with BT sparsity

In [ ]:
model.encoder.transformer.layers.enable_nested_tensor = True

In [ ]:
print("slow path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=False) as prof:
  for i in range(ITERATIONS):
    output = model(model_input)
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cpu_time_total", row_limit=5))

print("fast path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=False) as prof:
  with torch.no_grad():
    for i in range(ITERATIONS):
      output = model(model_input)
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cpu_time_total", row_limit=5))



#Measure DEVICE performance with slow and fast path, without and with sparsity

Please ensure that the runtime has GPUs enabled to see the performance benefits of Better Transformer fastpath execution on GPUs. You can confirm and change the Runtime type in the Google Colab menu with (Runtime > Change Runtime Type)

In [ ]:
model.to(DEVICE)
model.eval()
model_input = model_input.to(DEVICE)

### DEVICE performance without BT sparsity

In [ ]:
model.encoder.transformer.layers.enable_nested_tensor=False

In [ ]:
print("slow path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
  for i in range(ITERATIONS):
    output = model(model_input)
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=5))

print("fast path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
  with torch.no_grad():
    for i in range(ITERATIONS):
      output = model(model_input)
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=5))

### DEVICE performance performance with BT sparsity

In [ ]:
model.encoder.transformer.layers.enable_nested_tensor = True

In [ ]:
model.to(DEVICE)
model_input = model_input.to(DEVICE)

print("slow path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
  for i in range(ITERATIONS):
    output = model(model_input)
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=5))

print("fast path:")
print("==========")
with torch.autograd.profiler.profile(use_cuda=True) as prof:
  with torch.no_grad():
    for i in range(ITERATIONS):
      output = model(model_input)
print(prof.key_averages(group_by_stack_n=5).table(sort_by="self_cuda_time_total", row_limit=5))